# Análise de dados de tipos de estrela

Neste projeto, iremos a analisar uma base de dados de características de estrelas coletados pela NASA. 
A base está disponível [no Kaggle](https://www.kaggle.com/brsdincer/star-type-classification), começaremos observando o que a base nos fornece de dados, e então testaremos os algoritmos de classificação para achar o que melhor ajuda a prever o tipo de estrela baseado nos demais atributos e por fim mostraremos alguns gráficos baseados nos atributos e como elas se diferem em si.

Primeiramento vamos importar as bibliotecas necessárias e importar os dados:

In [12]:
# discriminante linear e quadrático
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis

# naive bayes, com distribuição gaussiana
from sklearn.naive_bayes import GaussianNB

# faz uma transformação de escala nos atributos, para que tenham média zero e desvio padrão 1
from sklearn.preprocessing import StandardScaler

# carregamento de dados artificiais do sitem OpenML
from sklearn.datasets import make_moons, make_circles, fetch_openml

from sklearn.metrics import f1_score, roc_auc_score, cohen_kappa_score, make_scorer, confusion_matrix, roc_curve, ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.metrics.scorer import roc_auc_scorer, accuracy_scorer

#Avaliação cruzada
from sklearn.model_selection import cross_validate, train_test_split, StratifiedKFold

import matplotlib.pyplot as plt
from matplotlib import colors

import numpy as np
import pandas as pd
from collections import Counter

In [17]:
#buscando os dados
data = pd.read_csv('Stars.csv')

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,Red,M,0
1,3042,0.000500,0.1542,16.60,Red,M,0
2,2600,0.000300,0.1020,18.70,Red,M,0
3,2800,0.000200,0.1600,16.65,Red,M,0
4,1939,0.000138,0.1030,20.06,Red,M,0


A seguir temos os dados um resumo dos primeiros dados da base:

In [18]:
data.head()

,Temperature,L,R,A_M,Color,Spectral_Class,Type
0,3068,0.002400,0.1700,16.12,Red,M,0
1,3042,0.000500,0.1542,16.60,Red,M,0
2,2600,0.000300,0.1020,18.70,Red,M,0
3,2800,0.000200,0.1600,16.65,Red,M,0
4,1939,0.000138,0.1030,20.06,Red,M,0


Agora vamos ver rapidamente os dados categoricos, começando pelas cores das estrelas:

In [24]:
print(pd.unique(data["Color"]))

['Red' 'Blue White' 'White' 'Yellowish White' 'Blue white'
 'Pale yellow orange' 'Blue' 'Blue-white' 'Whitish' 'yellow-white'
 'Orange' 'White-Yellow' 'white' 'yellowish' 'Yellowish' 'Orange-Red'
 'Blue-White']


Sobre a classe espectral, pode se ver o que eles significam [aqui] (https://en.wikipedia.org/wiki/Asteroid_spectral_types)

Abaixo os diferentes tipos espectrais das estrelas da base:

In [25]:
print(pd.unique(data["Spectral_Class"]))

['M' 'B' 'A' 'F' 'O' 'K' 'G']


Apesar da coluna "Type" aparentar ser numérica ela é categórica, pois os códigos (de 0 a 5) representam o tipo de estrela e o critério usado é puramente por conta do autor, os códigos estão descritos abaixo:

In [27]:
#    Red Dwarf - 0
#    Brown Dwarf - 1
#    White Dwarf - 2
#    Main Sequence - 3
#    Super Giants - 4
#    Hyper Giants - 5

print(pd.unique(data["Type"]))

[0 1 2 3 4 5]


Vamos começar compararando dados categóricos com outros categóricos, comparando o tipo espectral (`Spectral_Class`) com o tipo da estrela (`Type`):

In [30]:
#Aqui vem o código com o plot dos gráficos comparando para cada tipo de estrela (eixo y) como se distribui nos
# espectros (eixo x), ou seja, para cada tipo diferente de estrela ("Type") um gráfico de barra diferente, por 
# exemplo, quantas "Red Dwarf" (zero) temos no espectro 'O', 'M', 'K', 'G' e assim por diante

Agora vamos ver como é a distribuição da temperatura das estrelas para cada tipo:

In [31]:
#Aqui será um trecho de código que pegamos aquele gráfico de distruição e faremos gráficos para cada tipo de 
# estrela mas dessa vez o tipo será o eixo X, e a temperatura será o eixo Y do gráfico, aí poderíamos tirar 
# também a média, mediana, desvio padrão, etc da temperatura de cada tipo de estrela

Agora fazendo da mesma forma que vimos em aula vamos comparar os algoritmos de classficação e vê qual deles nos serve melhor classificar as estrelas:

In [ ]:
#Primeiro precisamos tratar os dados aqui, os atributos categóricos como Spectral_Class e Color devem ser 
# convertidos para valores numéricos (talvez eu não tenho certeza, pode ser também que seja melhor descartá-los 
# para prever o tipo de estrela e considerar somente os atributos núméricos)

In [28]:
#definindo os classificadores que testaremos 
classificadores = {
    'Naive Bayes (Gaussian)' : GaussianNB(),
    'Discriminante Linear' : LinearDiscriminantAnalysis(),
    'Discriminante Quadrático' : QuadraticDiscriminantAnalysis()
}

#critérios de avaliação
scoring = {'accuracy': accuracy_scorer,
           'f1': make_scorer(f1_score, pos_label='2'),
           'kappa' : make_scorer(cohen_kappa_score),
           'auc_roc' : roc_auc_scorer
}


results = []

for clf_name, clf in classificadores.items():
    tmp = cross_validate(clf, data, data["Type"], cv=10, scoring=scoring, return_train_score=True)
    tmp['clf'] = [clf_name]*10
    tmp['fold'] = range(10)

    results.append(pd.DataFrame(tmp))

results = pd.concat(results)
results.rename(columns={'fit_time' : 'train_time' , 'score_time' : 'test_time'}, inplace=True)

on.py", line 72, in inner_f
    return f(**kwargs)
  File "/home/vitorguilhermeantunes/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 795, in check_X_y
    X = check_array(X, accept_sparse=accept_sparse,
  File "/home/vitorguilhermeantunes/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 72, in inner_f
    return f(**kwargs)
  File "/home/vitorguilhermeantunes/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py", line 598, in check_array
    array = np.asarray(array, order=order, dtype=dtype)
  File "/home/vitorguilhermeantunes/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.py", line 83, in asarray
    return array(a, dtype, copy=False, order=order)
  File "/home/vitorguilhermeantunes/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py", line 1781, in __array__
    return np.asarray(self._values, dtype=dtype)
  File "/home/vitorguilhermeantunes/anaconda3/lib/python3.8/site-packages/numpy/core/_asarray.p